In [1]:
import numpy as np
import pandas as pd
import string
import re

In [2]:
df = (pd.read_csv("/Users/ahmedabukar/Downloads/:/data/alice-in-wonderland.txt",
                sep="\t")
      .squeeze()
      .str.split()
     )
df

0       [This, eBook, is, for, the, use, of, anyone, a...
1       [almost, no, restrictions, whatsoever., You, m...
2       [re-use, it, under, the, terms, of, the, Proje...
3       [with, this, eBook, or, online, at, www.gutenb...
4                         [Title:, Alice, in, Wonderland]
                              ...                        
1237                           [http://www.gutenberg.org]
1238    [This, Web, site, includes, information, about...
1239    [including, how, to, make, donations, to, the,...
1240    [Archive, Foundation,, how, to, help, produce,...
1241    [subscribe, to, our, email, newsletter, to, he...
Name: The Project Gutenberg EBook of Alice in Wonderland, by Lewis Carroll, Length: 1242, dtype: object

In [3]:
# what are the 10 most common words in the book?
df.explode().value_counts().head(10)

The Project Gutenberg EBook of Alice in Wonderland, by Lewis Carroll
the     732
and     363
a       321
to      311
of      299
in      210
she     197
was     160
said    129
you     122
Name: count, dtype: int64

In [4]:
# does this change if count the words without regard to case?
df.explode().str.lower().value_counts().head(10)

The Project Gutenberg EBook of Alice in Wonderland, by Lewis Carroll
the    795
and    381
a      326
to     318
of     313
she    233
in     223
was    160
you    150
it     140
Name: count, dtype: int64

In [5]:
# Does this change if you remove all the punctuations from the begining and ending?
df = df.explode().reindex()

def remove_punctuation(val):
    if val[0] in string.punctuation:
        val = val[1:]

    if len(val) > 1:
        if val[-1] in string.punctuation:
            suffix = val[-1]
            val = val[:-1]

    return val



In [6]:
df = df.reset_index(drop=True)

In [7]:
df.apply(remove_punctuation).value_counts().head(10)

The Project Gutenberg EBook of Alice in Wonderland, by Lewis Carroll
the      734
and      384
a        322
to       319
of       300
in       213
she      200
was      166
Alice    163
it       160
Name: count, dtype: int64

In [8]:
# how many capitalized words does the book contain?
df.loc[df.str.istitle()].count()

1524

In [9]:
# if you ignore punctuation and quotes before the start of a word, h
# how many capitalized words does the book contain? -- 1524, no change
(
    df
    .str
    .lstrip(string.punctuation)
    .str
    .istitle()
    .sum()
)

1524

In [10]:
# count the number of vowels in each word.
# what is the average number of vowels per word?
def count_vowels(val):
    pattern = re.compile(r"[aeiou]")

    return len(pattern.findall(val))

df.apply(count_vowels).mean()

1.5788111668757843

In [11]:
# remove the punctuation from start and end and the 
df.str.strip(string.punctuation).value_counts().head(10)

The Project Gutenberg EBook of Alice in Wonderland, by Lewis Carroll
the      735
and      384
a        322
to       320
of       302
in       213
she      201
was      167
Alice    165
it       164
Name: count, dtype: int64

In [12]:
(
    df
    .loc[df.str.strip(string.punctuation )
    .str

    .contains(r'^[A-Z]\w*$', regex=True)]
.count() 
)


1679

In [13]:
# what is the mean of all integers in Alice?
df.loc[df.str.isdigit()].astype(int).mean()

766.8333333333334

In [14]:
s = df[df.str.contains(r"\d+", regex=True)].str.strip(string.punctuation + string.ascii_letters)

In [15]:
s.str.replace(string.ascii_letters + string.punctuation, "").iloc[22]

'1.E.3'

In [16]:
pd.to_numeric(s, errors="coerce").dropna().mean()

6023.444444444444

In [19]:
#what words in Alice don't appear in the dictionary?
# and what are the five most common words?
filename = "/Users/ahmedabukar/Downloads/words_dictionary/words.txt"
dictionary = pd.Series(open(filename).read().split())

df.loc[~(df.isin(dictionary))].value_counts().head()

The Project Gutenberg EBook of Alice in Wonderland, by Lewis Carroll
Project         77
Gutenberg-tm    53
She             36
Alice,          29
Alice.          27
Name: count, dtype: int64

In [35]:
# what are the minimum and maximum number of words per paragraph?
# identify paragraph points in the alice book
s = pd.Series(open(filename).read().split("\n\n"))
s.str.split().str.len().describe()

count         1.0
mean     235886.0
std           NaN
min      235886.0
25%      235886.0
50%      235886.0
75%      235886.0
max      235886.0
dtype: float64